In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_validate
import matplotlib.pyplot as plt

In [34]:
testSize = 0.25
numberOfLayers = 4
k = 10

In [35]:
X = pd.read_csv("../binary/X.csv",header=None).values
y = pd.read_csv("../binary/y.csv",header=None,squeeze=True).values
X_toClassify = pd.read_csv("../binary/XToClassify.csv",header=None).values
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize)


X_train.shape

(108, 921)

In [52]:
NNC = MLPClassifier(hidden_layer_sizes=(numberOfLayers,),activation='relu',early_stopping=True)
scores = cross_validate(NNC,X_train,y_train,cv=k,scoring='accuracy')
print(scores['test_score'],scores['test_score'].mean())
# NNC.fit(X_train,y_train)

[ 1.   0.5  1.   1.   1.   1.   1.   1.   1.   1. ] 0.95


In [30]:
accuracy_score(y_test,NNC.predict(X_test))

0.54166666666666663

In [31]:
pd.DataFrame(NNC.predict(X_toClassify)).to_csv("../binary/PredictedClasses.csv",index=False)